In [1]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt
import zipfile

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/dogs-vs-cats')

In [3]:
os.mkdir('/kaggle/working/dogs-vs-cats/train/cats')
os.mkdir('/kaggle/working/dogs-vs-cats/train/dogs')

os.mkdir('/kaggle/working/dogs-vs-cats/validation')
os.mkdir('/kaggle/working/dogs-vs-cats/validation/cats')
os.mkdir('/kaggle/working/dogs-vs-cats/validation/dogs')

In [4]:
fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join('/kaggle/working/dogs-vs-cats/train', fname)
    dst = os.path.join('/kaggle/working/dogs-vs-cats/train/cats', fname)
    shutil.move(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join('/kaggle/working/dogs-vs-cats/train', fname)
    dst = os.path.join('/kaggle/working/dogs-vs-cats/validation/cats', fname)
    shutil.move(src, dst)

In [5]:
fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join('/kaggle/working/dogs-vs-cats/train', fname)
    dst = os.path.join('/kaggle/working/dogs-vs-cats/train/dogs', fname)
    shutil.move(src, dst)
       
fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join('/kaggle/working/dogs-vs-cats/train', fname)
    dst = os.path.join('/kaggle/working/dogs-vs-cats/validation/dogs', fname)
    shutil.move(src, dst)

In [91]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [92]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),
             metrics=['acc'])

In [93]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),
             metrics=['acc'])
model.summary()

# Data Generators

In [94]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [95]:
train_generator = train_datagen.flow_from_directory('/kaggle/working/dogs-vs-cats/train',
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')

In [96]:
validation_generator = val_datagen.flow_from_directory('/kaggle/working/dogs-vs-cats/validation',
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

# Model fitting and Accuracy/Loss Evaluation

In [97]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50# for fast validating, there will be some randomness in results.
)

In [98]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.xticks(np.arange(10))
plt.legend()

In [99]:
# median accuracy for training
median = np.median(history.history['acc'])

# standard deviation of training loss
std = np.std(history.history['loss'])

print (median, std)

# Data Augmentation

In [100]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

In [101]:
train_generator = train_datagen.flow_from_directory('/kaggle/working/dogs-vs-cats/train',
                                                    target_size=(150, 150), 
                                                    batch_size=32, 
                                                    class_mode='binary')


In [102]:
validation_generator = test_datagen.flow_from_directory(
    '/kaggle/working/dogs-vs-cats/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

# Model Fitting (augmented) and Accuracy/Loss Evaluation

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
#mean of validation loss
np.mean(history.history['val_loss'])

In [ ]:
# mean of validation accuracy for the last 5 epochs
np.mean(history.history['val_acc'][5:10])

In [ ]:
model.evaluate(validation_generator)

# The End of Homework

In [72]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [61]:
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [62]:
model1 = define_model()

#datagen = ImageDataGenerator(featurewise_center=True)
# specify imagenet mean values for centering
#datagen.mean = [123.68, 116.779, 103.939]


In [25]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# prepare iterator
train_it = train_datagen.flow_from_directory('/kaggle/working/dogs-vs-cats/train',
    class_mode='binary', batch_size=64, target_size=(224, 224))

In [26]:
val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    '/kaggle/working/dogs-vs-cats/validation',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [63]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [28]:
history = model.fit(train_it, steps_per_epoch=len(train_it), epochs=10, validation_data=val_ds, callbacks=[checkpoint])

In [29]:
model.save('final_model.h5')

In [37]:
hist = history.history
plt.plot(hist['val_accuracy'], label='val')
plt.plot(hist['accuracy'], label='train')

plt.legend()

In [39]:
hist = history.history
plt.plot(hist['val_loss'], label='val_loss')
plt.plot(hist['loss'], label='loss')

plt.legend()

# TO DO

### Define CNN_NEW Model (droprated added)

In [84]:
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    drop = Dropout(0.25)(class1)
    output = Dense(1, activation='sigmoid')(drop)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [85]:
model1 = define_model()
train_datagen = ImageDataGenerator(rescale=1./255)
# prepare iterator
train_it = train_datagen.flow_from_directory('/kaggle/working/dogs-vs-cats/train',
    class_mode='binary', batch_size=64, target_size=(224, 224))

val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    '/kaggle/working/dogs-vs-cats/validation',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [86]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v2_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [87]:
history4 = model1.fit(train_it, steps_per_epoch=len(train_it), epochs=16, validation_data=val_ds, callbacks=[checkpoint])

In [40]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/dogs-vs-cats/test')

In [49]:
test_gen = ImageDataGenerator(rescale=1./255)

test_ds = test_gen.flow_from_directory(
    '/kaggle/working/dogs-vs-cats/test',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [89]:
model1.evaluate(test_ds)